# Visual Chatbot App with ChatGPT, LangChain and Streamlit

In [2]:
!pip install langchain==0.1.12 --quiet
!pip install langchain-openai==0.0.8 --quiet
!pip install langchain-community==0.0.29 --quiet
!pip install streamlit==1.32.2 --quiet
!pip install pyngrok==7.1.5 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.

In [23]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPEN_API_KEY')
os.environ['NGROK_AUTH_TOKEN'] = userdata.get('NGROK_AUTH_TOKEN')


## App Code

In [46]:
%%writefile app.py
import streamlit as st
import openai
import io
import base64
from PIL import Image
import os

# OpenAI API Key (Replace with your API key or set it as an environment variable)
openai.api_key = os.environ['OPENAI_API_KEY']

# Streamlit App Configuration
st.set_page_config(page_title="Visual Chatbot", page_icon="🤖")
st.title("🤖 Visual Chatbot")

# File uploader for an optional image
uploaded_file = st.file_uploader("Upload an image (optional)", type=["png", "jpg", "jpeg"])

img_base64 = None
if uploaded_file:
    # Display the uploaded image
    image = Image.open(uploaded_file)
    st.image(image, caption="Uploaded Image", use_column_width=True)
    # Convert image to Base64 string
    img_bytes = io.BytesIO()
    image.save(img_bytes, format="PNG")
    img_base64 = base64.b64encode(img_bytes.getvalue()).decode("utf-8")

# User input for a question
user_question = st.text_input("Ask your question:")

if user_question:
    # Build the messages list
    messages = [
        {"role": "system", "content": "You are a helpful assistant."}
    ]
    if img_base64:
        # If an image is uploaded, include both text and image in the user message
        messages.append({
            "role": "user",
            "content": [
                {"type": "text", "text": user_question},
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{img_base64}"}}
            ]
        })
    else:
        # No image: use a simple text message
        messages.append({"role": "user", "content": user_question})

    # Call GPT-4o using the new client interface
    response = openai.chat.completions.create(
        model="gpt-4o",  # Replace with your desired multi-modal model if needed
        messages=messages,
        max_tokens=150
    )

    # Extract and display the response using dot notation
    ai_response = response.choices[0].message.content
    st.subheader("🤖 AI Response:")
    st.write(ai_response)

# Footer
st.markdown("---")
st.caption("🔍 Powered by OpenAI GPT-4o & Streamlit")


Overwriting app.py


In [47]:
!streamlit run app.py --server.port=8989 &>./logs.txt &

In [15]:
from pyngrok import ngrok
import yaml

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken
# Get your authtoken from `ngrok_credentials.yml` file
# with open('ngrok_credentials.yml', 'r') as file:
#     NGROK_AUTH_TOKEN = yaml.safe_load(file)
ngrok.set_auth_token(os.environ['NGROK_AUTH_TOKEN'] )

# Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8989)
print("Streamlit App:", ngrok_tunnel.public_url)

Streamlit App: https://8e17-35-243-149-155.ngrok-free.app
